In [671]:
# Imports
import pandas as pd 
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

In [724]:
#Load and analyze features
training = pd.read_csv('Training_DataSet.csv')

In [725]:
## Check to see if any listing prices or vehicle trim are missing 
empty_cells_in_price = training['Dealer_Listing_Price'].isna() | (training['Dealer_Listing_Price'] == '')
empty_cells_in_trim = training['Vehicle_Trim'].isna() | (training['Vehicle_Trim'] == '')

## Drop rows with empty listing prices and trim as we cant learn from these
rows_to_drop = empty_cells_in_price | empty_cells_in_trim
training = training[~rows_to_drop].reset_index(drop=True)


In [726]:
## City change cities to high or low population
## These cities are expensive and probably have higher car prices
top_16_us_cities_by_population = [
    "New York City",
    "Los Angeles",
    "Chicago",
    "Houston",
    "Phoenix",
    "Philadelphia",
    "San Antonio",
    "San Diego",
    "Dallas",
    "San Jose",
    "Austin",
    "Jacksonville",
    "Fort Worth",
    "Columbus",
    "Charlotte",
    "Indianapolis"
]
training['high_population'] = training['SellerCity'].apply(lambda x: 1 if x in top_16_us_cities_by_population else 0)

In [727]:
#Dealing with seller information
## We will only keep state and a high low city population indicator to avoid overfitting
training = training.drop(columns=['SellerCity', 'SellerZip'])

##Convert states into booleans which will allow XGboost to work with these features.
training = pd.get_dummies(training, columns=['SellerState'])

In [728]:
## Seller List SRC
training['SellerListSrc'].value_counts()

SellerListSrc
Digital Motorworks (DMi)    3043
Inventory Command Center    1180
HomeNet Automotive          1029
Jeep Certified Program       544
My Dealer Center              29
Sell It Yourself              14
Name: count, dtype: int64

In [729]:
## Check SellerListSrc for NA and replace values then convert to dummy variables
training.loc[training['SellerListSrc'].isna()]
training['SellerListSrc'] = training['SellerListSrc'].fillna('Sell It Yourself')
training = pd.get_dummies(training, columns=['SellerListSrc'])

In [730]:
## SellerIsPriv
## Only 14 Trues this will not provide that much information and potentially cause overfitting
## remove this column 
training['SellerIsPriv'].sum()
training = training.drop(columns=['SellerIsPriv'])

In [731]:
## SellerName
## Most of the names have the type of the car in it
## We can delete this as this information is contained in VehMake
training['SellerName'].head()
training = training.drop(columns=['SellerName'])

In [732]:
##Vehicle Information
## VehBodyStyle 
## only SUV will not play a factor remove
training['VehBodystyle'].value_counts()
training = training.drop(columns=['VehBodystyle'])

In [733]:
##Vehicle certified
##Certified already in true and false form 
training['VehCertified'].value_counts()

VehCertified
False    4820
True     1021
Name: count, dtype: int64

In [734]:
##Vehicle Exterior, interior
training['VehColorExt'].value_counts()
training['VehColorInt'].value_counts()

training['VehColorExt'].fillna('Unknown', inplace=True)
training['VehColorInt'].fillna('Unknown', inplace=True)

#Convert to simpler color groups
color_groups = {
    'Black': ['black'],
    'White': ['white'],
    'Silver': ['silver', 'gray', 'grey'],
    'Blue': ['blue'],
    'Red': ['red'],
    'Green': ['green'],
    'Yellow': ['yellow', 'gold'],
    'Brown': ['brown', 'tan'],
    'Orange': ['orange'],
    'Purple': ['purple'],
    'Pink': ['pink'],
    'Tan': ['tan'],
    'Beige': ['beige'],
    'Metallic': ['metallic'],
    'Clearcoat': ['clearcoat']
}

def map_color(color, color_groups):
    color = color.lower()  
    for standard_color, keywords in color_groups.items():
        if any(keyword in color for keyword in keywords):
            return standard_color
    return 'Unknown' 

## Apply the mapping function to standardize colors
training['VehColorExt_Standardized'] = training['VehColorExt'].apply(map_color, color_groups=color_groups)
training['VehColorInt_Standardized'] = training['VehColorInt'].apply(map_color, color_groups=color_groups)

## Convert to dummies
training = pd.get_dummies(training, columns=['VehColorExt_Standardized'])
training = pd.get_dummies(training, columns=['VehColorInt_Standardized'])

## Drop redundant columns
training = training.drop(columns=['VehColorExt'])
training = training.drop(columns=['VehColorInt'])

In [735]:
## Vehicle Drive Train
training['VehDriveTrain'].value_counts()

training['VehDriveTrain'].fillna('Unknown', inplace=True)

## keywords for drive type groups
drive_type_groups = {
    '4WD': ['4wd', '4x4', 'four wheel drive'],
    'FWD': ['fwd', 'front wheel drive', 'front-wheel drive'],
    'AWD': ['awd', 'all wheel drive', 'all-wheel drive', 'allwheeldrive'],
    '2WD': ['2wd']
}

def map_drive_type(drive_type, drive_type_groups):
    drive_type = drive_type.lower()  # Convert to lower case for easier matching
    for standard_drive_type, keywords in drive_type_groups.items():
        if any(keyword in drive_type for keyword in keywords):
            return standard_drive_type
    return 'Unknown'  

## Apply the mapping function to standardize drive types
training['DriveType_Standardized'] = training['VehDriveTrain'].apply(map_drive_type, drive_type_groups=drive_type_groups)

## Convert to dummies
training = pd.get_dummies(training, columns=['DriveType_Standardized'])

## Drop redundant columns
training = training.drop(columns=['VehDriveTrain'])

In [736]:
##VehEngine
training['VehEngine'].fillna('Unknown', inplace=True)

## Define keywords for engine groups
engine_groups = {
    '3.6L V6': ['3.6l v6'],
    '3.0L V6': ['3.0l v6'],
    'Regular Unleaded V-6': ['regular unleaded v-6'],
    'Gas V6': ['gas v6'],
    'V6 Natural Aspiration': ['v6 natural aspiration'],
    'V6 Flex Fuel': ['flex fuel', 'v6, flex fuel'],
    '6 Cylinder': ['6 cylinder'],
    '5.7L': ['5.7l']
}

def map_engine(engine, engine_groups):
    engine = engine.lower()  
    for standard_engine, keywords in engine_groups.items():
        if any(keyword in engine for keyword in keywords):
            return standard_engine
    return 'Unknown'  

## Apply the mapping function to standardize engines
training['VehEngine_Standardized'] = training['VehEngine'].apply(map_engine, engine_groups=engine_groups)

## Dummies
training = pd.get_dummies(training, columns=['VehEngine_Standardized'])

## Drop redundant columns
training = training.drop(columns=['VehEngine'])

In [737]:
## Veh Feats
## Most data covered in the other categories already drop
training = training.drop(columns=['VehFeats'])

In [738]:
## VehFuel
training['VehFuel'].fillna('Unknown', inplace=True)
training = pd.get_dummies(training, columns=['VehFuel'])

In [739]:
## VehHistory 
## Condense to accidents or not
training['VehHistory'].fillna('Unknown', inplace=True)

# Function to categorize based on "Accident"
def categorize_accident(status):
    if 'accident' in status.lower():
        return 'Accident'
    else:
        return 'No Accident'

## Apply the function to create a new column
training['AccidentCategory'] = training['VehHistory'].apply(categorize_accident)

training = pd.get_dummies(training, columns=['AccidentCategory'])

## Drop redundant columns
training = training.drop(columns=['VehHistory'])


In [740]:
## VehListDays
training['VehListdays'].fillna(training['VehListdays'].median(), inplace = True)

In [741]:
##VehMake
training['VehMake'].fillna('Unknown', inplace=True)
training = pd.get_dummies(training, columns=['VehMake'])

In [742]:
##VehMileage
training['VehMileage'].fillna(training['VehMileage'].median(), inplace = True)

In [743]:
## VehModel
## Same As vehicle make 
training = training.drop(columns=['VehModel'])

In [744]:
## VehPriceLabel
## This could implement bias going to remove
training = training.drop(columns=['VehPriceLabel'])

In [745]:
## VehSellerNotes
## Most information is already contained in the other features
training = training.drop(columns=['VehSellerNotes'])

In [746]:
## VehType
## Every car is used can remove
training = training.drop(columns=['VehType'])

In [747]:
##VehTransmision
## Mostly everything is automatic can remove as well
training = training.drop(columns=['VehTransmission'])

In [748]:
## VehYear
## Convert to dummies
training = pd.get_dummies(training, columns=['VehYear'])

In [749]:
ListingID = training['ListingID']
VehTrim = training['Vehicle_Trim']
Price = training['Dealer_Listing_Price']
training = training.drop(columns=['ListingID', 'Vehicle_Trim', 'Dealer_Listing_Price'])

## Remove columns not in test data
train_columns = ['SellerState_HI', 'VehColorInt_Standardized_Blue', 'SellerState_OR', 'SellerState_VT', 
                 'VehColorExt_Standardized_Purple', 'VehColorExt_Standardized_Yellow', 'SellerState_ME', 
                 'VehColorExt_Standardized_Pink', 'SellerState_MT', 'VehFuel_Unknown', 'SellerState_NM', 
                 'DriveType_Standardized_2WD']

training = training.drop(columns = train_columns)

In [750]:
#Modeling Price
## Convert all boolean columns to integers
bool_cols = training.select_dtypes(include=['bool']).columns
training[bool_cols] = training[bool_cols].astype(int)

## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(training, Price, test_size=0.2, random_state=42)

## Convert data to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


## Define parameters for the XGBoost model
params = {
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'eta': 0.1,
    'eval_metric': 'rmse'
}

## Train the model
num_round = 100
bst = xgb.train(params, dtrain, num_round)

## Make predictions
preds = bst.predict(dtest)

## Evaluate the model
mse = mean_squared_error(y_test, preds)
rmse = mse ** 0.5
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 3845.2123946382235


In [751]:
#Modeling Trim
categories = {
    'Limited': 0,
    'Premium Luxury': 1,
    'Laredo': 2,
    'Luxury': 3,
    'Overland': 4,
    'Altitude': 5,
    'Summit': 6,
    'Trailhawk': 7,
    'Base': 8,
    'Platinum': 9,
    'High Altitude': 10,
    'SRT': 11,
    'FWD': 12,
    'Luxury FWD': 13,
    'Laredo E': 14,
    'Premium Luxury FWD': 15,
    'Trackhawk': 16,
    'Sterling Edition': 17,
    'Luxury AWD': 18,
    'Platinum AWD': 19,
    'Premium Luxury AWD': 20,
    '75th Anniversary': 21,
    'Limited 75th Anniversary Edition': 22,
    'SRT Night': 23,
    'Upland': 24,
    'Limited 4x4': 25,
    '75th Anniversary Edition': 26,
    'Limited 75th Anniversary': 27
}

## Apply mapping
training['Trim'] = VehTrim.map(categories)
training['Trim'].fillna(0, inplace=True)

## Define features and target
y = training['Trim']
X = training.drop('Trim', axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_classifier = SVC(kernel='rbf', decision_function_shape='ovr', random_state=42)

svm_classifier.fit(X_train_scaled, y_train)

y_pred = svm_classifier.predict(X_test_scaled)

##Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")



Accuracy: 0.5560307955517536


In [752]:
# Apply all to test data
test = pd.read_csv('Test_DataSet.csv')

top_16_us_cities_by_population = [
    "New York City",
    "Los Angeles",
    "Chicago",
    "Houston",
    "Phoenix",
    "Philadelphia",
    "San Antonio",
    "San Diego",
    "Dallas",
    "San Jose",
    "Austin",
    "Jacksonville",
    "Fort Worth",
    "Columbus",
    "Charlotte",
    "Indianapolis"
]
test['high_population'] = test['SellerCity'].apply(lambda x: 1 if x in top_16_us_cities_by_population else 0)

##Dealing with seller information
## We will only keep state and a high low city population indicator to avoid overfitting
test = test.drop(columns=['SellerCity', 'SellerZip'])

##Convert states into booleans which will allow XGboost to work with these features.
test = pd.get_dummies(test, columns=['SellerState'])

## Check SellerListSrc for NA and replace values then convert to dummy variables
test.loc[test['SellerListSrc'].isna()]
test['SellerListSrc'] = test['SellerListSrc'].fillna('Sell It Yourself')
test = pd.get_dummies(test, columns=['SellerListSrc'])

## SellerIsPriv
## Only 14 Trues this will not provide that much information and potentially cause overfitting
## remove this column 
test = test.drop(columns=['SellerIsPriv'])

## SellerName
## Most of the names have the type of the car in it
## We can delete this as this information is contained in VehMake
test = test.drop(columns=['SellerName'])

##Vehicle Information
## VehBodyStyle 
## only SUV will not play a factor remove
test = test.drop(columns=['VehBodystyle'])


test['VehColorExt'].fillna('Unknown', inplace=True)
test['VehColorInt'].fillna('Unknown', inplace=True)

##Convert to simpler color groups
color_groups = {
    'Black': ['black'],
    'White': ['white'],
    'Silver': ['silver', 'gray', 'grey'],
    'Blue': ['blue'],
    'Red': ['red'],
    'Green': ['green'],
    'Yellow': ['yellow', 'gold'],
    'Brown': ['brown', 'tan'],
    'Orange': ['orange'],
    'Purple': ['purple'],
    'Pink': ['pink'],
    'Tan': ['tan'],
    'Beige': ['beige'],
    'Metallic': ['metallic'],
    'Clearcoat': ['clearcoat']
}

def map_color(color, color_groups):
    color = color.lower()  
    for standard_color, keywords in color_groups.items():
        if any(keyword in color for keyword in keywords):
            return standard_color
    return 'Unknown' 

## Apply the mapping function to standardize colors
test['VehColorExt_Standardized'] = test['VehColorExt'].apply(map_color, color_groups=color_groups)
test['VehColorInt_Standardized'] = test['VehColorInt'].apply(map_color, color_groups=color_groups)

##Convert to dummies
test = pd.get_dummies(test, columns=['VehColorExt_Standardized'])
test = pd.get_dummies(test, columns=['VehColorInt_Standardized'])

##Drop redundant columns
test = test.drop(columns=['VehColorExt'])
test = test.drop(columns=['VehColorInt'])

## Vehicle Drive Train
test['VehDriveTrain'].fillna('Unknown', inplace=True)

##keywords for drive type groups
drive_type_groups = {
    '4WD': ['4wd', '4x4', 'four wheel drive'],
    'FWD': ['fwd', 'front wheel drive', 'front-wheel drive'],
    'AWD': ['awd', 'all wheel drive', 'all-wheel drive', 'allwheeldrive'],
    '2WD': ['2wd']
}

def map_drive_type(drive_type, drive_type_groups):
    drive_type = drive_type.lower()  # Convert to lower case for easier matching
    for standard_drive_type, keywords in drive_type_groups.items():
        if any(keyword in drive_type for keyword in keywords):
            return standard_drive_type
    return 'Unknown'  

## Apply the mapping function to standardize drive types
test['DriveType_Standardized'] = test['VehDriveTrain'].apply(map_drive_type, drive_type_groups=drive_type_groups)

##Convert to dummies
test = pd.get_dummies(test, columns=['DriveType_Standardized'])

##Drop redundant columns
test = test.drop(columns=['VehDriveTrain'])

##VehEngine
test['VehEngine'].fillna('Unknown', inplace=True)

## Define keywords for engine groups
engine_groups = {
    '3.6L V6': ['3.6l v6'],
    '3.0L V6': ['3.0l v6'],
    'Regular Unleaded V-6': ['regular unleaded v-6'],
    'Gas V6': ['gas v6'],
    'V6 Natural Aspiration': ['v6 natural aspiration'],
    'V6 Flex Fuel': ['flex fuel', 'v6, flex fuel'],
    '6 Cylinder': ['6 cylinder'],
    '5.7L': ['5.7l']
}

def map_engine(engine, engine_groups):
    engine = engine.lower()  
    for standard_engine, keywords in engine_groups.items():
        if any(keyword in engine for keyword in keywords):
            return standard_engine
    return 'Unknown'  # If no keyword matches, categorize as 'Other'

## Apply the mapping function to standardize engines
test['VehEngine_Standardized'] = test['VehEngine'].apply(map_engine, engine_groups=engine_groups)

## Dummies
test = pd.get_dummies(test, columns=['VehEngine_Standardized'])

##Drop redundant columns
test = test.drop(columns=['VehEngine'])

## Veh Feats
## Most data covered in the other categories already drop
test = test.drop(columns=['VehFeats'])

## VehFuel
test['VehFuel'].fillna('Unknown', inplace=True)
test = pd.get_dummies(test, columns=['VehFuel'])

## VehHistory 
## Condense to accidents or not
test['VehHistory'].fillna('Unknown', inplace=True)

# Function to categorize based on "Accident"
def categorize_accident(status):
    if 'accident' in status.lower():
        return 'Accident'
    else:
        return 'No Accident'

## Apply the function to create a new column
test['AccidentCategory'] = test['VehHistory'].apply(categorize_accident)

test = pd.get_dummies(test, columns=['AccidentCategory'])

##Drop redundant columns
test = test.drop(columns=['VehHistory'])

## VehListDays
test['VehListdays'].fillna(test['VehListdays'].median(), inplace = True)

##VehMake
test['VehMake'].fillna('Unknown', inplace=True)
test = pd.get_dummies(test, columns=['VehMake'])

##VehMileage
test['VehMileage'].fillna(test['VehMileage'].median(), inplace = True)

## VehModel
## Same As vehicle make 
test = test.drop(columns=['VehModel'])

## VehPriceLabel
## This could implement bias going to remove
test = test.drop(columns=['VehPriceLabel'])

## VehSellerNotes
## Most information is already contained in the other features
test = test.drop(columns=['VehSellerNotes'])

## VehType
## Every car is used can remove
test = test.drop(columns=['VehType'])

##VehTransmision
## Mostly everything is automatic can remove as well
test = test.drop(columns=['VehTransmission'])

## VehYear
## Convert to dummies
test = pd.get_dummies(test, columns=['VehYear'])

test = test.drop(columns=['SellerListSrc_Cadillac Certified Program'])

In [753]:
#Predictions
ListingID_test = test['ListingID']
test = test.drop(columns=['ListingID'])

In [756]:
# Price Predictions
## Make predictions
dnewtest = xgb.DMatrix(test)
preds = bst.predict(dnewtest)


AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [758]:
# Category Predictions
X_test_scaled = scaler.transform(test)
y_pred = svm_classifier.predict(X_test_scaled)

In [759]:
## Answers
ANS = pd.DataFrame({
    'ListingID': ListingID_test,
    'Trim': y_pred,
    'Price': preds
})

In [761]:
##Change Answers to string
categories = {
    'Limited': 0,
    'Premium Luxury': 1,
    'Laredo': 2,
    'Luxury': 3,
    'Overland': 4,
    'Altitude': 5,
    'Summit': 6,
    'Trailhawk': 7,
    'Base': 8,
    'Platinum': 9,
    'High Altitude': 10,
    'SRT': 11,
    'FWD': 12,
    'Luxury FWD': 13,
    'Laredo E': 14,
    'Premium Luxury FWD': 15,
    'Trackhawk': 16,
    'Sterling Edition': 17,
    'Luxury AWD': 18,
    'Platinum AWD': 19,
    'Premium Luxury AWD': 20,
    '75th Anniversary': 21,
    'Limited 75th Anniversary Edition': 22,
    'SRT Night': 23,
    'Upland': 24,
    'Limited 4x4': 25,
    '75th Anniversary Edition': 26,
    'Limited 75th Anniversary': 27
}

## Create the inverse mapping
inverse_categories = {v: k for k, v in categories.items()}

ANS['Trim'] = training['Trim'].map(inverse_categories)

In [768]:
##save predictions
file_path = r'C:\Users\samsn\OneDrive\Boeing\Boeing Data Science Challenge Problem\output.csv'

ANS.to_csv(file_path, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ListingID  1000 non-null   int64  
 1   Trim       1000 non-null   object 
 2   Price      1000 non-null   float32
dtypes: float32(1), int64(1), object(1)
memory usage: 19.7+ KB
